In [ ]:
# NB-0: 경로/옵션
import os, sqlite3, json, re, glob
import pandas as pd
from pathlib import Path

DB_PATH   = "marryroute.db"      # 생성될 SQLite 파일
CSV_DIR   = "."                   # CSV 폴더 (필요시 절대경로로 변경)
CSV_GLOB  = "MarryRouteDB_*.csv"  # 파일 패턴
ENCODING  = "utf-8"               # 윈도우/엑셀 저장이면 "cp949" 권장

# (선택) 실제 파일 목록 확인
sorted(glob.glob(os.path.join(CSV_DIR, CSV_GLOB)))

In [ ]:
# NB-1: 기존 DB 삭제(완전 초기화)
p = Path(DB_PATH)
if p.exists():
    p.unlink()
print("DB 삭제:", not p.exists())

In [ ]:
# NB-2: 스키마 생성
SCHEMA_SQL = """
PRAGMA foreign_keys = ON;

CREATE TABLE IF NOT EXISTS vendor (
  vendor_id   INTEGER PRIMARY KEY AUTOINCREMENT,
  type        TEXT NOT NULL,      -- hall|studio|dress|makeup
  name        TEXT NOT NULL,      -- 업체명(conm)
  region      TEXT,               -- 지역/역명(subway)
  min_price   INTEGER,            -- 최소가(원) - CSV min_fee
  notes       TEXT
);

CREATE TABLE IF NOT EXISTS offering (
  offering_id  INTEGER PRIMARY KEY AUTOINCREMENT,
  vendor_id    INTEGER NOT NULL REFERENCES vendor(vendor_id) ON DELETE CASCADE,
  category     TEXT NOT NULL,     -- hall_package|studio_shoot|dress_rental|makeup_package
  package_name TEXT,              -- ex) allday|manager|meal_expense...
  price        INTEGER,           -- 원(KRW)
  meta_json    TEXT               -- JSON
);

CREATE INDEX IF NOT EXISTS idx_vendor_type       ON vendor(type);
CREATE INDEX IF NOT EXISTS idx_vendor_min_price  ON vendor(min_price);
CREATE INDEX IF NOT EXISTS idx_offering_vendor   ON offering(vendor_id);
CREATE INDEX IF NOT EXISTS idx_offering_cat      ON offering(category);
"""
with sqlite3.connect(DB_PATH) as conn:
    conn.executescript(SCHEMA_SQL)
print("스키마 생성 완료:", DB_PATH)

In [ ]:
# NB-3: 숫자/스케일/컬럼 유틸
def to_number(v):
    if pd.isna(v): return None
    s = str(v).strip().replace(",", "")
    if s in ("", "-"): return None
    m = re.findall(r"[0-9]+(?:\.[0-9]+)?", s)
    return float(m[0]) if m else None

def decide_scale(series):
    # 중앙값<500이면 '만원'으로 보고 ×10,000
    vals = [to_number(x) for x in series if not pd.isna(x)]
    vals = [v for v in vals if v is not None]
    if not vals: return 1
    vals.sort()
    med = vals[len(vals)//2]
    return 10000 if med < 500 else 1

def pick_col(columns, candidates):
    low = {str(c).strip().lower(): c for c in columns}
    for key in candidates:
        if key in low: return low[key]
    return None

In [ ]:
# NB-4: DB 헬퍼
def upsert_vendor(conn, vtype, name, region=None, min_price=None, notes=None):
    cur = conn.execute("SELECT vendor_id FROM vendor WHERE type=? AND name=?", (vtype, name))
    row = cur.fetchone()
    if row:
        vid = row[0]
        conn.execute(
            "UPDATE vendor SET region=COALESCE(?,region), min_price=COALESCE(?,min_price), notes=COALESCE(?,notes) WHERE vendor_id=?",
            (region, min_price, notes, vid)
        )
        return vid
    cur = conn.execute(
        "INSERT INTO vendor(type,name,region,min_price,notes) VALUES (?,?,?,?,?)",
        (vtype, name, region, min_price, notes)
    )
    return cur.lastrowid

def insert_offering(conn, vendor_id, category, package_name, price, meta=None):
    conn.execute(
        "INSERT INTO offering(vendor_id,category,package_name,price,meta_json) VALUES (?,?,?,?,?)",
        (vendor_id, category, package_name, price, json.dumps(meta or {}, ensure_ascii=False))
    )

In [ ]:
# NB-5: 카테고리별 임포터
EXCLUDE_FOR_MIN = {"fitting_fee","helper","snapphoto","snapvideo"}  # 옵션 제외

def import_wedding_hall(conn, df):
    name_col   = pick_col(df.columns, ["conm","name","hall_name","unnamed: 0"])
    region_col = pick_col(df.columns, ["subway","region"])
    min_col    = pick_col(df.columns, ["min_fee","minprice","min_price"])
    price_cols = [c for c in ["hall_rental_fee","meal_expense","snapphoto","snapvideo"] if c in df.columns]
    scale = decide_scale(pd.concat([df[c] for c in price_cols + ([min_col] if min_col else [])], axis=0)) if price_cols or min_col else 1

    season_col = pick_col(df.columns, ["season","season(t/f)","seaon(t/f)"])
    peak_col   = pick_col(df.columns, ["peak","peak(t/f)"])
    guar_col   = next((c for c in df.columns if "guarantor" in str(c).lower()), None)

    mapping = {
        "hall_rental_fee": ("hall_package", "hall_rental_fee"),
        "meal_expense":    ("hall_package", "meal_expense"),
        "snapphoto":       ("photo_option", "snapphoto"),
        "snapvideo":       ("video_option", "snapvideo"),
    }

    for _, r in df.iterrows():
        name = str(r[name_col]).strip() if name_col and not pd.isna(r[name_col]) else None
        if not name: continue
        region = str(r[region_col]).strip() if region_col and not pd.isna(r[region_col]) else None
        min_price = None
        if min_col and not pd.isna(r[min_col]):
            mv = to_number(r[min_col]); min_price = int(round(mv*scale)) if mv is not None else None

        vid = upsert_vendor(conn, "hall", name, region=region, min_price=min_price)

        meta = {}
        if season_col: meta["season"] = str(r[season_col]).strip()
        if peak_col:   meta["peak"]   = str(r[peak_col]).strip()
        if guar_col and not pd.isna(r[guar_col]): meta["num_guarantors"] = int(to_number(r[guar_col]))

        for col, (cat, pkg) in mapping.items():
            if col in df.columns:
                p = to_number(r[col]); price = int(round(p*scale)) if p is not None else None
                insert_offering(conn, vid, cat, pkg, price, meta)

def import_studio(conn, df):
    name_col   = pick_col(df.columns, ["conm","name","studio","unnamed: 0"])
    region_col = pick_col(df.columns, ["subway","region"])
    min_col    = pick_col(df.columns, ["min_fee","minprice","min_price"])
    cands      = [("std_price","std"), ("afternoon_price","afternoon"), ("allday_price","allday")]
    price_cols = [c for c,_ in cands if c in df.columns]
    scale = decide_scale(pd.concat([df[c] for c in price_cols + ([min_col] if min_col else [])], axis=0)) if price_cols or min_col else 1

    for _, r in df.iterrows():
        name = str(r[name_col]).strip() if name_col and not pd.isna(r[name_col]) else None
        if not name: continue
        region = str(r[region_col]).strip() if region_col and not pd.isna(r[region_col]) else None
        min_price = None
        if min_col and not pd.isna(r[min_col]):
            mv = to_number(r[min_col]); min_price = int(round(mv*scale)) if mv is not None else None

        vid = upsert_vendor(conn, "studio", name, region=region, min_price=min_price)
        for col, pkg in cands:
            if col in df.columns:
                p = to_number(r[col]); price = int(round(p*scale)) if p is not None else None
                insert_offering(conn, vid, "studio_shoot", pkg, price, {})

def import_wedding_dress(conn, df):
    name_col   = pick_col(df.columns, ["conm","name","dessshop_name","dressshop_name","unnamed: 0"])
    region_col = pick_col(df.columns, ["subway","region"])
    min_col    = pick_col(df.columns, ["min_fee","minprice","min_price"])
    cands      = [("wedding","wedding"), ("photo","photo"), ("wedding+photo","wedding+photo"),
                  ("fitting_fee","fitting_fee"), ("helper","helper")]
    price_cols = [c for c,_ in cands if c in df.columns]
    scale = decide_scale(pd.concat([df[c] for c in price_cols + ([min_col] if min_col else [])], axis=0)) if price_cols or min_col else 1

    for _, r in df.iterrows():
        name = str(r[name_col]).strip() if name_col and not pd.isna(r[name_col]) else None
        if not name: continue
        region = str(r[region_col]).strip() if region_col and not pd.isna(r[region_col]) else None
        min_price = None
        if min_col and not pd.isna(r[min_col]):
            mv = to_number(r[min_col]); min_price = int(round(mv*scale)) if mv is not None else None

        vid = upsert_vendor(conn, "dress", name, region=region, min_price=min_price)
        for col, pkg in cands:
            if col in df.columns:
                p = to_number(r[col]); price = int(round(p*scale)) if p is not None else None
                insert_offering(conn, vid, "dress_rental", pkg, price, {})

def import_makeup(conn, df):
    name_col   = pick_col(df.columns, ["conm","name","shop","brand","unnamed: 0"])
    region_col = pick_col(df.columns, ["subway","region"])
    min_col    = pick_col(df.columns, ["min_fee","minprice","min_price"])
    role_cols  = [(c, c.split("(")[0].strip().lower()) for c in df.columns if any(k in str(c).lower() for k in ["manager","vicedirector","director"])]
    price_cols = [c for c,_ in role_cols]
    scale = decide_scale(pd.concat([df[c] for c in price_cols + ([min_col] if min_col else [])], axis=0)) if price_cols or min_col else 1

    for _, r in df.iterrows():
        name = str(r[name_col]).strip() if name_col and not pd.isna(r[name_col]) else None
        if not name: continue
        region = str(r[region_col]).strip() if region_col and not pd.isna(r[region_col]) else None
        min_price = None
        if min_col and not pd.isna(r[min_col]):
            mv = to_number(r[min_col]); min_price = int(round(mv*scale)) if mv is not None else None

        vid = upsert_vendor(conn, "makeup", name, region=region, min_price=min_price)
        for col, role in role_cols:
            p = to_number(r[col])
            if p is None: continue
            m = re.search(r"\((\d+)\)", str(col))
            meta = {"role": role}
            if m: meta["variant"] = m.group(1)
            price = int(round(p*scale))
            insert_offering(conn, vid, "makeup_package", role, price, meta)

In [ ]:
# NB-6: CSV 일괄 임포트
csv_files = sorted(glob.glob(os.path.join(CSV_DIR, CSV_GLOB)))
assert csv_files, f"CSV가 없습니다. 확인: {CSV_DIR}/{CSV_GLOB}"

with sqlite3.connect(DB_PATH) as conn:
    conn.execute("PRAGMA foreign_keys = ON;")
    # 재적재 시 초기화
    conn.execute("DELETE FROM offering;")
    conn.execute("DELETE FROM vendor;")

    for path in csv_files:
        try:
            df = pd.read_csv(path, encoding=ENCODING)
        except Exception:
            df = pd.read_csv(path, engine="python", encoding=ENCODING)

        low = path.lower()
        cols = [c.lower() for c in df.columns]
        if "wedding_hall" in low or "hall_rental_fee" in cols:
            import_wedding_hall(conn, df);  print(f"[{Path(path).name}] hall OK")
        elif "studio" in low or "allday_price" in cols:
            import_studio(conn, df);        print(f"[{Path(path).name}] studio OK")
        elif "wedding_dress" in low or "wedding+photo" in cols:
            import_wedding_dress(conn, df); print(f"[{Path(path).name}] dress OK")
        elif "makeup" in low or any(("manager" in c or "director" in c) for c in cols):
            import_makeup(conn, df);        print(f"[{Path(path).name}] makeup OK")
        else:
            print(f"[{Path(path).name}] 스킵(타입 판별 실패)")
    conn.commit()

with sqlite3.connect(DB_PATH) as conn:
    v = conn.execute("SELECT COUNT(*) FROM vendor").fetchone()[0]
    o = conn.execute("SELECT COUNT(*) FROM offering").fetchone()[0]
print(f"✅ Import 완료 | vendors={v}, offerings={o}")

In [ ]:
# NB-7: 조회(최저가: vendor.min_price 우선 → 없으면 옵션 제외 계산)
EXCLUDE_FROM_MIN = ("fitting_fee", "helper", "snapphoto", "snapvideo")

def find_vendors(db_path, vtype=None, region=None, price_max=None, keyword=None, limit=10):
    base = f"""
    SELECT
        v.vendor_id, v.type, v.name, v.region,
        COALESCE(
          v.min_price,
          MIN(CASE
                WHEN o.package_name IS NULL
                     OR LOWER(o.package_name) NOT IN ({",".join(["?"]*len(EXCLUDE_FROM_MIN))})
                THEN o.price
              END)
        ) AS min_price,
        COUNT(o.offering_id) AS cnt_offers,
        CASE WHEN v.min_price IS NOT NULL THEN 'vendor' ELSE 'computed' END AS min_source
    FROM vendor v
    LEFT JOIN offering o ON o.vendor_id = v.vendor_id
    WHERE 1=1
    """
    params = [*EXCLUDE_FROM_MIN]
    if vtype:  base += " AND v.type=?"; params.append(vtype)
    if region: base += " AND IFNULL(v.region,'') LIKE ?"; params.append(f"%{region}%")
    if keyword:base += " AND v.name LIKE ?"; params.append(f"%{keyword}%")
    base += " GROUP BY v.vendor_id, v.type, v.name, v.region, v.min_price"

    sql = f"SELECT * FROM ({base}) t"
    if price_max is not None:
        sql += " WHERE t.min_price IS NULL OR t.min_price<=?"
        params.append(price_max)
    sql += " ORDER BY (t.min_price IS NULL), t.min_price, t.name LIMIT ?"
    params.append(limit)

    with sqlite3.connect(db_path) as c:
        c.row_factory = sqlite3.Row
        return [dict(r) for r in c.execute(sql, params).fetchall()]

def get_offerings(db_path, vendor_id):
    sql = """
    SELECT offering_id, category, package_name, price, meta_json
    FROM offering
    WHERE vendor_id=?
    ORDER BY (price IS NULL), price, category, package_name
    """
    with sqlite3.connect(db_path) as c:
        c.row_factory = sqlite3.Row
        rows = c.execute(sql, (vendor_id,)).fetchall()
        out = []
        for r in rows:
            d = dict(r)
            d["meta_json"] = json.loads(d["meta_json"]) if d["meta_json"] else {}
            out.append(d)
        return out

def fmt_price(v):
    if v is None: return "-"
    s = f"{v/10000.0:.1f}".rstrip("0").rstrip(".")
    return f"{s}만원"

In [ ]:
# NB-8: 추천/상세 테스트
rows = find_vendors(DB_PATH, vtype="dress", price_max=int(200*10000), limit=5)
for i, r in enumerate(rows, 1):
    src = "표준(min_fee)" if r["min_source"] == "vendor" else "계산"
    print(f"{i:>2}. #{r['vendor_id']} [{r['type']}] {r['name']} ({r.get('region') or '-'})"
          f" | 최저가 {fmt_price(r['min_price'])} [{src}] | 상품 {r['cnt_offers']}개")

if rows:
    vid = rows[0]["vendor_id"]
    print(f"\n[상세] {rows[0]['name']}  (#{vid})")
    for o in get_offerings(DB_PATH, vid):
        print(f"- [{o['category']}] {o['package_name']}: {fmt_price(o['price'])} {o['meta_json']}")

In [ ]:
# NB-9: 노트북용 간단 데모(입력→추천→상세)
VTYPES = ["hall", "studio", "dress", "makeup"]

def run_demo(db_path=DB_PATH, limit=10):
    print("=== 메리루트 | 노트북 데모 === (빈값=건너뛰기, q=종료)")
    while True:
        vtype = input(f"\n유형 [{', '.join(VTYPES)}]: ").strip()
        if vtype.lower() == "q": break
        if vtype and vtype not in VTYPES:
            print("⚠️ 유형 오류"); continue
        region = input("지역(예: 강남, 홍대): ").strip()
        if region.lower() == "q": break
        price_s = input("예산 상한(만원, 예: 120 또는 5.5): ").strip()
        if price_s.lower() == "q": break
        price_max = int(round(float(price_s)*10000)) if price_s else None
        keyword = input("키워드(업체명 일부): ").strip()
        if keyword.lower() == "q": break

        rows = find_vendors(db_path, vtype or None, region or None, price_max, keyword or None, limit)
        if not rows:
            print("검색 결과 없음"); continue

        for i, r in enumerate(rows, 1):
            src = "표준(min_fee)" if r["min_source"] == "vendor" else "계산"
            print(f"{i:>2}. #{r['vendor_id']} [{r['type']}] {r['name']} ({r.get('region') or '-'})"
                  f" | 최저가 {fmt_price(r['min_price'])} [{src}] | 상품 {r['cnt_offers']}개")

        sel = input("\n상세 번호(엔터=다시검색, q=종료): ").strip()
        if sel.lower() == "q": break
        if not sel: continue
        if not sel.isdigit() or not (1 <= int(sel) <= len(rows)):
            print("⚠️ 번호 오류"); continue

        chosen = rows[int(sel)-1]
        items = get_offerings(db_path, chosen["vendor_id"])
        print(f"\n[상세] {chosen['name']}  (#{chosen['vendor_id']})")
        if not items:
            print("오퍼링 없음")
        else:
            for it in items:
                meta = f" {it['meta_json']}" if it["meta_json"] else ""
                print(f"- [{it['category']}] {it['package_name']}: {fmt_price(it['price'])}{meta}")

# 실행
run_demo()